<a href="https://colab.research.google.com/github/michael-borck/weatherwise-template/blob/main/starter_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌦️ WeatherWise – Starter Notebook

Welcome to your **WeatherWise** project notebook! This scaffold is designed to help you build your weather advisor app using Python, visualisations, and AI-enhanced development.

---

📄 **Full Assignment Specification**  
See [`ASSIGNMENT.md`](ASSIGNMENT.md) or check the LMS for full details.

📝 **Quick Refresher**  
A one-page summary is available in [`resources/assignment-summary.md`](resources/assignment-summary.md).

---

🧠 **This Notebook Structure is Optional**  
You’re encouraged to reorganise, rename sections, or remove scaffold cells if you prefer — as long as your final version meets the requirements.

✅ You may delete this note before submission.



## 🧰 Setup and Imports

This section imports commonly used packages and installs any additional tools used in the project.

- You may not need all of these unless you're using specific features (e.g. visualisations, advanced prompting).
- The notebook assumes the following packages are **pre-installed** in the provided environment or installable via pip:
  - `requests`, `matplotlib`, `pyinputplus`
  - `fetch-my-weather` (for accessing weather data easily)
  - `hands-on-ai` (for AI logging, comparisons, or prompting tools)

If you're running this notebook in **Google Colab**, uncomment the following lines to install the required packages.


In [1]:
# 🧪 Optional packages — uncomment if needed in Colab or JupyterHub
!pip install fetch-my-weather
!pip install hands-on-ai


   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------- ----------------------- 0.8/2.0 MB 4.2 MB/s eta 0:00:01
   ---------------------------------------- 2.0/2.0 MB 6.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/16.6 MB ? eta -:--:--
    --------------------------------------- 0.3/16.6 MB ? eta -:--:--
   -- ------------------------------------- 1.0/16.6 MB 3.6 MB/s eta 0:00:05
   -------- ------------------------------- 3.7/16.6 MB 7.5 MB/s eta 0:00:02
   ------------------------- -------------- 10.5/16.6 MB 15.9 MB/s eta 0:00:01
   ---------------------------------------- 16.6/16.6 MB 21.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   ---------------------------------------- 3.8/3.8 MB 75.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ------------------------- -------------- 1.0/1.7 MB 52.4 MB/s eta 0:00:01
   --------------------------------

In [ ]:
import os

#  Testing ai api key: student-api-key-123
os.environ['HANDS_ON_AI_SERVER'] = 'http://ollama.serveur.au'
os.environ['HANDS_ON_AI_MODEL'] = 'granite3.2'
os.environ['HANDS_ON_AI_API_KEY'] = input('Enter your API key: ')

## 📦 Setup and Configuration
Import required packages and setup environment.

In [ ]:
import requests
import matplotlib.pyplot as plt
import pyinputplus as pyip
# ✅ Import after installing (if needed)
from fetch_my_weather import get_weather
from hands_on_ai.chat import get_response

# Add any other setup code here

import json

from datetime import datetime, timedelta

## 🌤️ Weather Data Functions

In [ ]:
# Define get_weather_data() function here
def safe_int(value, default=0):
    """Safely convert value to int, return default if conversion fails."""
    if value is None:
        return default
    try:
        return int(value)
    except (ValueError, TypeError):
        return default

def safe_float(value, default=0.0):
    """Safely convert value to float, return default if conversion fails."""
    if value is None:
        return default
    try:
        return float(value)
    except (ValueError, TypeError):
        return default

def safe_get_attr(obj, attr_path, default=None):
    """Safely get nested attribute, return default if any step fails."""
    try:
        attrs = attr_path.split('.')
        result = obj
        for attr in attrs:
            if hasattr(result, attr):
                result = getattr(result, attr)
            else:
                return default
        return result
    except:
        return default



# Update the weather data processing section in get_weather_data()
def get_weather_data(location, forecast_days=5):
    """
    Updated version with better error handling
    """
    if not isinstance(forecast_days, int):
        try:
            forecast_days = int(forecast_days)
            if not (1 <= forecast_days <= 5):
                print(f"Warning: forecast_days ({forecast_days}) out of range (1-5). Clamping.")
        except ValueError:
            print(f"Warning: forecast_days ('{forecast_days}') is not a valid integer. Defaulting to 1.")
            forecast_days = 1

    if forecast_days < 1:
        forecast_days = 1
    elif forecast_days > 5:
        forecast_days = 5

    processed_data = {"current": None, "forecast": [], "error": None}

    try:
        raw_data = get_weather(location)
        
        if raw_data is None:
            processed_data["error"] = f"No data returned from weather service for '{location}'."
            return processed_data
        
        # Check for errors
        if hasattr(raw_data, 'error') and getattr(raw_data, 'error'):
            processed_data["error"] = raw_data.error
            return processed_data

        # Process current conditions with safe conversions
        if hasattr(raw_data, 'current_condition') and raw_data.current_condition:
            current = raw_data.current_condition[0]
            processed_data["current"] = {
                "temperature_c": safe_int(getattr(current, 'temp_C', None)),
                "feels_like_c": safe_int(getattr(current, 'FeelsLikeC', None)),
                "description": safe_get_attr(current, 'weatherDesc.0.value', "N/A"),
                "humidity_percent": safe_int(getattr(current, 'humidity', None)),
                "wind_kmph": safe_int(getattr(current, 'windspeedKmph', None)),
                "precipitation_mm": safe_float(getattr(current, 'precipMM', None))
            }
        else:
            processed_data["error"] = f"Current conditions data missing or malformed for '{location}'."
            return processed_data

        # Process forecast with safe conversions
        if hasattr(raw_data, 'weather') and raw_data.weather:
            full_forecast_list = raw_data.weather
            for i, day_data in enumerate(full_forecast_list):
                if i >= forecast_days:
                    break
                
                hourly_slots = getattr(day_data, 'hourly', [])
                representative_hourly_data = None
                if len(hourly_slots) > 4:
                    representative_hourly_data = hourly_slots[4]
                elif hourly_slots:
                    representative_hourly_data = hourly_slots[0]
                
                if representative_hourly_data:
                    forecast_entry = {
                        "date": getattr(day_data, 'date', 'N/A'),
                        "max_temp_c": safe_int(getattr(day_data, 'maxtempC', None)),
                        "min_temp_c": safe_int(getattr(day_data, 'mintempC', None)),
                        "avg_temp_c": safe_int(getattr(day_data, 'avgtempC', None)),
                        "description": safe_get_attr(representative_hourly_data, 'weatherDesc.0.value', "N/A"),
                        "chance_of_rain_percent": safe_int(getattr(representative_hourly_data, 'chanceofrain', None)),
                        "total_precip_mm": safe_float(getattr(representative_hourly_data, 'precipMM', None))
                    }
                    processed_data["forecast"].append(forecast_entry)
        
        if not processed_data["current"] and not processed_data["forecast"] and not processed_data["error"]:
             processed_data["error"] = f"Failed to parse valid weather data for '{location}' despite receiving a response."

    except Exception as e:
        processed_data["error"] = f"An unexpected error occurred fetching/processing weather for '{location}': {str(e)}"
        
    return processed_data

## 📊 Visualisation Functions

In [ ]:
# Define create_temperature_visualisation() and create_precipitation_visualisation() here
def create_temperature_visualisation(weather_data, output_type='display'):
    """
    Create a line chart visualisation of daily maximum and minimum temperature data.

    Args:
        weather_data (dict): The processed weather data, expected to have a 'forecast'
                             key with a list of daily forecast dictionaries. Each
                             dictionary should contain 'date', 'max_temp_c', and
                             'min_temp_c'.
        output_type (str): Either 'display' to show the plot directly,
                             or 'figure' to return the matplotlib figure object.

    Returns:
        If output_type is 'figure', returns the matplotlib.figure.Figure object.
        Otherwise (for 'display'), displays the plot and returns None.
    """
    if weather_data.get("error"):
        print(f"Cannot create visualisation: {weather_data['error']}")
        if output_type == 'figure':
            # Return an empty figure or None if error, to maintain return type consistency
            fig, ax = plt.subplots()
            ax.text(0.5, 0.5, "Error: No data to display.", ha='center', va='center')
            return fig
        return None

    forecast_list = weather_data.get("forecast")
    if not forecast_list:
        print("No forecast data available to create temperature visualisation.")
        if output_type == 'figure':
            fig, ax = plt.subplots()
            ax.text(0.5, 0.5, "No forecast data.", ha='center', va='center')
            return fig
        return None

    dates = []
    max_temps = []
    min_temps = []

    for day_data in forecast_list:
        try:
            # Attempt to parse date for cleaner x-axis labels, fallback to string
            date_obj = datetime.strptime(day_data.get("date"), "%Y-%m-%d")
            dates.append(date_obj.strftime("%b %d")) # e.g., "Oct 27"
        except (ValueError, TypeError):
            dates.append(day_data.get("date", "Unknown")) # Fallback if date is not as expected
            
        max_temps.append(day_data.get("max_temp_c"))
        min_temps.append(day_data.get("min_temp_c"))

    # Filter out None values if any field was missing from a forecast entry
    # This requires dates, max_temps, and min_temps to be filtered consistently
    valid_indices = [i for i, (mx, mn) in enumerate(zip(max_temps, min_temps)) if mx is not None and mn is not None]
    
    if not valid_indices:
        print("Insufficient temperature data in forecast to create visualisation.")
        if output_type == 'figure':
            fig, ax = plt.subplots()
            ax.text(0.5, 0.5, "Insufficient data.", ha='center', va='center')
            return fig
        return None
        
    dates = [dates[i] for i in valid_indices]
    max_temps = [max_temps[i] for i in valid_indices]
    min_temps = [min_temps[i] for i in valid_indices]

    if not dates: # Double check after filtering
        print("No valid dates found for temperature visualisation after filtering.")
        if output_type == 'figure':
            fig, ax = plt.subplots()
            ax.text(0.5, 0.5, "No valid dates.", ha='center', va='center')
            return fig
        return None

    fig, ax = plt.subplots(figsize=(10, 6)) # Create a figure and an axes

    ax.plot(dates, max_temps, marker='o', linestyle='-', color='r', label='Max Temp (°C)')
    ax.plot(dates, min_temps, marker='s', linestyle='--', color='b', label='Min Temp (°C)')

    ax.set_xlabel("Date")
    ax.set_ylabel("Temperature (°C)")
    ax.set_title("Daily Temperature Forecast")
    ax.legend() # Show legend
    ax.grid(True, linestyle=':', alpha=0.7) # Add a grid for readability

    plt.xticks(rotation=45, ha="right") # Rotate date labels for better fit
    plt.tight_layout() # Adjust plot to prevent labels from being cut off

    if output_type == 'figure':
        return fig
    elif output_type == 'display':
        plt.show()
        return None
    else:
        print(f"Warning: Unknown output_type '{output_type}'. Displaying plot by default.")
        plt.show()
        return None


In [ ]:

def create_precipitation_visualisation(weather_data, output_type='display'):
    """
    Create a bar chart visualisation of daily chance of rain.

    Args:
        weather_data (dict): The processed weather data, expected to have a 'forecast'
                             key with a list of daily forecast dictionaries. Each
                             dictionary should contain 'date' and 'chance_of_rain_percent'.
        output_type (str): Either 'display' to show the plot directly,
                             or 'figure' to return the matplotlib figure object.

    Returns:
        If output_type is 'figure', returns the matplotlib.figure.Figure object.
        Otherwise (for 'display'), displays the plot and returns None.
    """
    if weather_data.get("error"):
        print(f"Cannot create precipitation visualisation: {weather_data['error']}")
        if output_type == 'figure':
            fig, ax = plt.subplots()
            ax.text(0.5, 0.5, "Error: No data to display.", ha='center', va='center')
            return fig
        return None

    forecast_list = weather_data.get("forecast")
    if not forecast_list:
        print("No forecast data available to create precipitation visualisation.")
        if output_type == 'figure':
            fig, ax = plt.subplots()
            ax.text(0.5, 0.5, "No forecast data.", ha='center', va='center')
            return fig
        return None

    dates = []
    chance_of_rain = []

    for day_data in forecast_list:
        try:
            date_obj = datetime.strptime(day_data.get("date"), "%Y-%m-%d")
            dates.append(date_obj.strftime("%b %d")) # e.g., "Oct 27"
        except (ValueError, TypeError):
            dates.append(day_data.get("date", "Unknown"))
            
        chance_of_rain.append(day_data.get("chance_of_rain_percent"))

    # Filter out None values for chance_of_rain, keeping dates and precip consistent
    valid_indices = [i for i, rain_chance in enumerate(chance_of_rain) if rain_chance is not None]
    
    if not valid_indices:
        print("Insufficient precipitation data in forecast to create visualisation.")
        if output_type == 'figure':
            fig, ax = plt.subplots()
            ax.text(0.5, 0.5, "Insufficient data.", ha='center', va='center')
            return fig
        return None

    dates = [dates[i] for i in valid_indices]
    chance_of_rain = [chance_of_rain[i] for i in valid_indices]
    
    if not dates: # Double check after filtering
        print("No valid dates found for precipitation visualisation after filtering.")
        if output_type == 'figure':
            fig, ax = plt.subplots()
            ax.text(0.5, 0.5, "No valid dates.", ha='center', va='center')
            return fig
        return None

    fig, ax = plt.subplots(figsize=(10, 6)) # Create a figure and an axes

    bars = ax.bar(dates, chance_of_rain, color='skyblue', label='Chance of Rain (%)')

    # Add text labels on top of each bar
    for bar in bars:
        yval = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2.0, yval + 1, f'{yval}%', ha='center', va='bottom')

    ax.set_xlabel("Date")
    ax.set_ylabel("Chance of Rain (%)")
    ax.set_title("Daily Precipitation Forecast (Chance of Rain)")
    ax.set_ylim(0, 105) # Set y-axis limit to slightly above 100% for text labels
    ax.legend()
    ax.grid(True, axis='y', linestyle=':', alpha=0.7) # Add a horizontal grid

    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()

    if output_type == 'figure':
        return fig
    elif output_type == 'display':
        plt.show()
        return None
    else:
        print(f"Warning: Unknown output_type '{output_type}'. Displaying plot by default.")
        plt.show()
        return None

## 🤖 Natural Language Processing

In [ ]:
# Define parse_weather_question() and generate_weather_response() here
def parse_weather_question(question):
    """
    Parse a natural language weather question.

    Args:
        question (str): User's weather-related question

    Returns:
        dict: Extracted information including location, time period, and weather attribute
    """
    pass

## 🧭 User Interface

In [ ]:
# Define menu functions using pyinputplus or ipywidgets here

## 🧩 Main Application Logic

In [ ]:
# Tie everything together here
def generate_weather_response(parsed_question, weather_data):
    """
    Generate a natural language response to a weather question.

    Args:
        parsed_question (dict): Parsed question data
        weather_data (dict): Weather data

    Returns:
        str: Natural language response
    """
    pass

## 🧪 Testing and Examples

In [ ]:
# Include sample input/output for each function

## 🗂️ AI Prompting Log (Optional)
Add markdown cells here summarising prompts used or link to AI conversations in the `ai-conversations/` folder.